<a href="https://colab.research.google.com/github/Redislabs-Solution-Architects/Redis-Workshops/blob/main/05-LangChain_Redis/05.03_Google_Gemini_LangChain_Redis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Document Question Answering with Langchain, Google Gemini and Redis

![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

This notebook would use your own deployment of VertexAI, Redis with Vector Similarity Search and LangChain to answer questions about the information contained in a document.

### Install Dependencies


In [1]:
# Update nltk version to latest. Must be the first cell to avoid restart
!pip install --upgrade -q nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 16.5 MB/s eta 0:00:00


In [2]:
!pip install -q redis langchain-community langchain-core langchain-google-genai unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 18.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.3/261.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.8/401.8 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.8/160.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 760.0/760.0 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 21.6 MB/s eta 0:00:00
  

### Authenticate notebook to Google Cloud API

You can get your Google Cloud API key at https://console.cloud.google.com/apis/credentials

For security reason we recomment to restrict the key to allow only `Generative Language API`

***Note - If you are participating in a workshop, your instructor should provide you with an API key.***

In [3]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Provide your Google API Key: ")

Provide your Google API Key: ··········


### [OPTIONAL] Install Redis Stack

Redis will be used as the Vector Similarity Search engine for Langchain. This installs the Redis Stack database for local usage if you cannot or do not want to create a Redis Cloud database (https://redis.io/try-free/), and it installs the `redis-cli` that will be used for checking database connectivity, etc.

In [4]:
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes

deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb jammy main
Starting redis-stack-server, database path /var/lib/redis-stack


### Setup Redis connection
***Note - Replace Redis connection values if using a Redis Cloud instance!***

In [5]:
import redis
import os

REDIS_HOST = os.getenv("REDIS_HOST", "localhost")
REDIS_PORT = os.getenv("REDIS_PORT", "6379")
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")
# Replace values above with your own if using Redis Cloud instance
# REDIS_HOST="redis-18374.c253.us-central1-1.gce.cloud.redislabs.com"
# REDIS_PORT=18374
# REDIS_PASSWORD="1TNxTEdYRDgIDKM2gDfasupCADXXXX"

# Shortcut for redis-cli $REDIS_CONN command
# If SSL is enabled on the endpoint add --tls
if REDIS_PASSWORD!="":
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT} -a {REDIS_PASSWORD} --no-auth-warning"
else:
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT}"

# If SSL is enabled on the endpoint, use rediss:// as the URL prefix
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"
INDEX_NAME = f"qna:idx"

# Test Redis connection
!redis-cli $REDIS_CONN PING

PONG


In [6]:
# Import Langchain dependencies
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.vectorstores.redis import Redis
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.chains import RetrievalQA

### Load text and split it into manageable chunks

Without this step any large body of text would exceed the limit of tokens you can feed to the LLM. Vector search will return the most relevant chunks in our database to include in our prompt.

In [7]:
# Add your own URLs here
urls = [
    "https://raw.githubusercontent.com/Redislabs-Solution-Architects/Redis-Workshops/main/resources/nvidia_q1fy25_earnings_call.txt",
    "https://raw.githubusercontent.com/Redislabs-Solution-Architects/Redis-Workshops/main/resources/nvidia_q4fy24_earnings_call.txt"
]
loader = UnstructuredURLLoader(urls=urls)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, add_start_index = True)
texts = text_splitter.split_documents(documents)

# Optionally examine the result of text load+splitting
# for text in texts:
#  print(text)

### Initialize embeddings engine

Here we are using Gemini embeddings engine.

In [8]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

### Initialize LLM

In this notebook we are using Gemini Pro LLM from Google Cloud.

In [9]:
llm = ChatGoogleGenerativeAI(model="gemini-pro")

### Create vector store from the documents using Redis as Vector Database

In [10]:
def get_vectorstore() -> Redis:
    """Create the Redis vectorstore."""

    try:
        vectorstore = Redis.from_existing_index(
            embedding=embeddings,
            index_name=INDEX_NAME,
            redis_url=REDIS_URL
        )
        return vectorstore
    except:
        pass

    # Load Redis with documents
    vectorstore = Redis.from_documents(
        documents=texts,
        embedding=embeddings,
        index_name=INDEX_NAME,
        redis_url=REDIS_URL
    )
    return vectorstore

redis = get_vectorstore()

## Specify the prompt template

PromptTemplate defines the exact text of the response that would be fed to the LLM. This step is optional, but the defaults usually work well for Gemini but might fall short for other models.

In [11]:
def get_prompt():
    """Create the QA chain."""
    from langchain.prompts import PromptTemplate
    from langchain.chains import RetrievalQA

    # Define our prompt
    prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, say that you don't know, don't try to make up an answer.

    This should be in the following format:

    Question: [question here]
    Answer: [answer here]

    Begin!

    Context:
    ---------
    {context}
    ---------
    Question: {question}
    Answer:"""

    prompt = PromptTemplate(
        template=prompt_template,
        input_variables=["context", "question"]
    )
    return prompt

### Putting it all together

This is where the Langchain brings all the components together in a form of a simple QnA chain

In [12]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=redis.as_retriever(search_type="similarity_distance_threshold",search_kwargs={"distance_threshold":0.5}),
    # return_source_documents=True,
    chain_type_kwargs={"prompt": get_prompt()},
    # verbose=True
)

### Debugging Redis

The code block below is example of how you can interact with the Redis Database

In [13]:
# !redis-cli $REDIS_CONN keys "*"
# !redis-cli $REDIS_CONN HGETALL "doc:qna:idx:2005b093c6af4f6c899779802910ed69"

## Finally - let's ask questions!

Examples:
- Who is Jensen Huang?
- How is the company diversifying?
- How is Nvidia involved in the data center business?

In [14]:
query = "Was Generative AI discussed, and if so, what was said?"
res=qa.invoke(query)
res['result']

"Yes, Generative AI was discussed. It was described as a whole new application space, a whole new way of doing computing, and a whole new industry that is being formed. It was also said that Generative AI is driving NVIDIA's growth."

#### Cleanup
(Optional) Delete all keys and indexes from the database.

In [15]:
###-- FLUSHDB will wipe out the entire database!!! Use with caution --###
# !redis-cli $REDIS_CONN flushdb